### 付録H　温水暖房用熱源機の最大能力

#### H.1 記号及び単位

省略

#### H.2 温水暖房用熱源機の最大能力

$$\large
q_{max,hs} = q_{rq,H} \times \sum_{i} A_{HCZ,i} \times f_{cT} \times f_{cI}
$$

$q_{max,hs}$：温水暖房用熱源機の最大能力(W)  
$q_{rq,H}$：単位面積当たりの必要暖房能力(W/m<sup>2</sup>)  
$A_{HCZ,i}$：暖冷房区画$i$の床面積(m<sup>2</sup>)  
$f_{cT}$：外気温度能力補正係数  
$f_{cI}$：間歇運転能力補正係数

外気温度補正係数$f_{cT}$は、1.05の値とする。

<div style="text-align: center;font-weight: bold;">表　単位面積当たりの必要暖房能力(W/m<sup>2</sup>)</div>

|地域の区分|（い）温水暖房の放熱器を主たる居室及びその他の居室に設置する場合|（ろ）温水暖房の放熱器を主たる居室に設置しその他の居室に設置しない場合|（は）温水暖房の放熱器を主たる居室に設置せずその他の居室に設置する場合|
|-|-|-|-|
|1|90.02|139.26|62.28|
|2|77.81|120.65|53.26|
|3|73.86|111.32|53.81|
|4|77.74|118.98|55.41|
|5|83.24|126.56|59.43|
|6|69.76|106.48|49.93|
|7|74.66|112.91|53.48|
|8|－|－|－|


<div style="text-align: center;font-weight: bold;">表　間歇運転能力補正係数</div>

|主たる居室の運転方法|その他の居室の運転方法|間歇運転能力補正係数|
|-|-|-|
|連続運転|連続運転|1.0|
|連続運転|間歇運転|1.0|
|間歇運転|連続運転|2.25|
|間歇運転|間歇運転|2.25|
|連続運転|放熱器を設置しない<sup>※</sup>|1.0|
|間歇運転|放熱器を設置しない<sup>※</sup>|3.03|
|放熱器を設置しない|連続運転|1.0|
|放熱器を設置しない|間歇運転|1.62|


※ その他の居室が無い場合も含む。 